<a href="https://colab.research.google.com/github/cwagner2325/WiseInbox_ML/blob/main/Wise_Inbox_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastapi
!pip install uvicorn
!pip install pickle5
!pip install pydantic
!pip install scikit-learn
!pip install requests
!pip install pypi-json
!pip install pyngrok
!pip install nest-asyncio
!pip install scikit-learn

In [2]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
import random
import os
from google.colab import userdata
from pyngrok import conf
from time import sleep
import sklearn
from sklearn.pipeline import Pipeline

In [10]:
app = FastAPI()

In [11]:
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [12]:
class model_input(BaseModel):

    Email : str
    Sender: str
    Subject : str

In [13]:
with open('/content/drive/MyDrive/XGBModel4', 'rb') as model_file:
    classifier = pickle.load(model_file)

with open('/content/drive/MyDrive/XGBModel4Vectorizer', 'rb') as vectorizer_file:
    vectorizer = pickle.load(vectorizer_file)

In [14]:
def predictEmail(input_text):
  transformed_email = vectorizer.transform([input_text])
  prediction = classifier.predict(transformed_email)
  probabilities = classifier.predict_proba(transformed_email)

  PHISHING = 1;
  SAFE = 0

  if prediction == SAFE:
      print("Predicted class: Safe Email")
      print("Probabilities:", probabilities.item(0))
      return "Safe Email", probabilities.item(0)
  elif prediction == PHISHING:
      print("Predicted class: Phishing Email")
      print("Probabilities:", probabilities.item(1))
      return "Phishing Email", probabilities.item(1)

In [15]:
@app.post('/email_prediction')
def email_prediction(input_parameters : model_input):
    input_data = input_parameters.json()
    input_dictionary = json.loads(input_data)

    emailBody = input_dictionary['Email']
    sender = input_dictionary['Sender']
    subject = input_dictionary['Subject']

    predictionResult, confidence = predictEmail(emailBody)

    if predictionResult == "Phishing Email":
        # If confidence is 1 - prediction is 50
        # If confidence is .7 - prediction is 68
        # If confidence is .5 - prediciton is 80
        prediction = 110 - (60 * confidence)
    else:
        prediction = 80 + (20 * confidence)

    return {"prediction": prediction}

In [ ]:
os.environ["TOKEN"] = userdata.get("TOKEN")
ngrok.set_auth_token(os.environ["TOKEN"])

ngrok_tunnel = ngrok.connect(8000)

# print('Public URL:', ngrok_tunnel.public_url)
# print("Here is a test command to run in a terminal: ")
# print('curl --insecure -X POST -H "Content-Type: application/json" -d \'{\n' +
#       '\t"Email": "example@example.com",\n\t"Sender": "John Doe",\n\t"Subject": "Important News" \n}\' ' +
#       ngrok_tunnel.public_url + "/email_prediction")

print("Running at public URL:", ngrok_tunnel.public_url + "/email_prediction")

nest_asyncio.apply()
uvicorn.run(app, port=8000, log_level="warning")

Running at public URL: https://a9a8-35-236-170-4.ngrok-free.app/email_prediction
Predicted class: Safe Email
Probabilities: 0.9286127686500549
Predicted class: Safe Email
Probabilities: 0.5999019742012024
Predicted class: Phishing Email
Probabilities: 0.7536976337432861
Predicted class: Safe Email
Probabilities: 0.9868502616882324
Predicted class: Phishing Email
Probabilities: 0.9643445014953613
Predicted class: Phishing Email
Probabilities: 0.5659196972846985
Predicted class: Safe Email
Probabilities: 0.8871576189994812
Predicted class: Phishing Email
Probabilities: 0.8821790814399719
Predicted class: Phishing Email
Probabilities: 0.8821790814399719
Predicted class: Phishing Email
Probabilities: 0.9643445014953613
Predicted class: Safe Email
Probabilities: 0.9927634000778198
Predicted class: Safe Email
Probabilities: 0.8871576189994812
Predicted class: Phishing Email
Probabilities: 0.7536976337432861
Predicted class: Phishing Email
Probabilities: 0.8324265480041504
